In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy.signal import convolve2d
import os

In [8]:
################################# Contour Detection ###########################################

# 读取输入图像
image = cv2.imread('Testset/Test10.png')

# 转换为灰度图像
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 边缘检测
edges = cv2.Canny(gray, 50, 150, apertureSize=7)

# 轮廓检测
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 筛选最外层的轮廓
outer_contour = max(contours, key=cv2.contourArea)
# print(outer_contour)
# print(len(outer_contour), outer_contour[0], outer_contour[1])


# 多边形逼近
epsilon = 0.02 * cv2.arcLength(outer_contour, True)
approx = cv2.approxPolyDP(outer_contour, epsilon, True)

# 绘制外边框
cv2.drawContours(gray, [approx], -1, (0, 0, 0), 2)
# cv2.rectangle(image, (minx, miny), (maxx, maxy), (0, 255,0))

# 显示结果
# cv2.namedWindow('Game Board Contour', cv2.WINDOW_NORMAL)
# cv2.imshow('Game Board Contour', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite("Game Board Contour.jpg",gray)

True

In [9]:
################################# Contour Detection and Perspecive Transform ###########################################
# load image
file_path ="Testset/Test10.png"
image = cv2.imread(file_path)

# Get the file name
file_name = os.path.basename(file_path)
name=file_name.split(".")[0]
# Print the file name
print("File name:", name)

# Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Canny Edge Detection
edges = cv2.Canny(gray, 50, 150, apertureSize=7)

# Contour detection
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# print(contours)
# Fetch the Outmost Contour
outer_contour = max(contours, key=cv2.contourArea)
# print(outer_contour)
# Assuming you have detected the outer contour and stored it in 'outer_contour'

# Calculate the convex hull of the outer contour
hull = cv2.convexHull(outer_contour)

# Extract the corner points from the convex hull
corner_points = hull.reshape(-1, 2)
print(corner_points)

# Get the four corner points

# Sort the corner points based on x coordinates
corner_points_sorted_x = sorted(corner_points, key=lambda point: point[0])

# Sort the corner points based on y coordinates
corner_points_sorted_y = sorted(corner_points, key=lambda point: point[1])

# # Find points with minimum x and minimum y
x_max= corner_points_sorted_x[-1][0]
# small_x_points= [i for i in corner_points if i[0]< x_max/2]
# sorted_y=  sorted(small_x_points, key=lambda point: point[1])
# x_min_y_min= min(sorted_y,key=lambda point: point[1])

# # Find points with minimum x and maximum y
x_min= corner_points_sorted_x[0][0]
# large_x_points= [i for i in corner_points if i[0]> x_max/2]
# sorted_y=  sorted(large_x_points, key=lambda point: point[1])
# x_min_y_max= min(sorted_y,key=lambda point: point[1])


# Find points with maximum x and minimum y
y_max= corner_points_sorted_y[-1][1]
y_min= corner_points_sorted_y[0][1]
small_y_points= [i for i in corner_points if i[1]< y_max/10+y_min]
sorted_x=  sorted(small_y_points, key=lambda point: point[0])
y_min_x_max= max(sorted_x,key=lambda point: point[0])

# Find points with minimum x and minimum y
# x_min_y_min= min(sorted_x,key=lambda point: point[0])
#####another way###########
min_dis=corner_points[0][0]*corner_points[0][0]+corner_points[0][1]*corner_points[0][1]
x_min_y_min = corner_points[0]
for point in corner_points:
    dis=point[0]*point[0]+point[1]*point[1]
    if dis<min_dis:
        min_dis=dis
        x_min_y_min=point
###########################

# Find points with maximum x and  maximum y
large_y_points=[i for i in corner_points if i[1]> y_max-y_max/10]
sorted_x=  sorted(large_y_points, key=lambda point: point[0])
# y_max_x_max= max(sorted_x,key=lambda point: point[0])
#####another way###########
max_dis=0
for point in corner_points:
    dis=point[0]*point[0]+point[1]*point[1]
    if dis>max_dis:
        max_dis=dis
        y_max_x_max=point
###########################
# Find points with minimum x and maximum y
x_min_y_max=min(sorted_x,key=lambda point: point[0])

# Display the four points
print("Point with minimum x and minimum y coordinates:", x_min_y_min)
print("Point with minimum x and maximum y coordinates:", x_min_y_max)
print("Point with maximum x and maximum y coordinates:", y_max_x_max)
print("Point with maximum x and minimum y coordinates:", y_min_x_max)



# Visualize the corner points for verification
for point in corner_points:
    cv2.circle(image, tuple(point), 5, (0, 0, 255), -1)

# Display the image with corner points
# cv2.imshow('Outer Contour with Corner Points', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite(f"test_img_Perspective_Transform/Corner_Points{name}.jpg", image)

new_corner_points = [x_min_y_min,y_min_x_max,y_max_x_max,x_min_y_max]
print(new_corner_points)
# Define the size of the output rectangle (desired width and height)
width = 1200
height = 1200

# Define the target points (top-left, top-right, bottom-right, bottom-left) for perspective transformation
pts_dst = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

# Convert the corner points to NumPy array
pts_src = np.array(new_corner_points, dtype=np.float32)

# Calculate the perspective transformation matrix
perspective_matrix = cv2.getPerspectiveTransform(pts_src, pts_dst)

# Apply the perspective transformation
result = cv2.warpPerspective(gray, perspective_matrix, (width, height))

# Display the result
cv2.imshow('Perspective Transformation', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite(f"perspective_transformed{name}.jpg", result)

File name: Test10
[[1344  751]
 [2321  753]
 [2327  754]
 [2329  755]
 [2332  758]
 [2335  764]
 [2353 1837]
 [2353 1860]
 [2352 1863]
 [2350 1866]
 [2348 1868]
 [2343 1870]
 [2340 1871]
 [2301 1871]
 [1231 1869]
 [1227 1867]
 [1223 1864]
 [1221 1860]
 [1220 1857]
 [1220 1841]
 [1233  768]
 [1234  759]
 [1236  755]
 [1238  753]
 [1243  751]]
Point with minimum x and minimum y coordinates: [1236  755]
Point with minimum x and maximum y coordinates: [1220 1857]
Point with maximum x and maximum y coordinates: [2350 1866]
Point with maximum x and minimum y coordinates: [2335  764]
[array([1236,  755], dtype=int32), array([2335,  764], dtype=int32), array([2350, 1866], dtype=int32), array([1220, 1857], dtype=int32)]


True

In [58]:
########## Test Binary Image Contour detection ##########
# load image
file_path ="Testset/Test7.png"

# Get the file name
file_name = os.path.basename(file_path)
name=file_name.split(".")[0]
# Print the file name
print("File name:", name)


rgb_img = cv2.imread(file_path)
gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
height, width = gray_img.shape

white_padding = np.zeros((50, width, 3))
white_padding[:, :] = [255, 255, 255]
rgb_img = np.row_stack((white_padding, rgb_img))

gray_img = 255 - gray_img
gray_img[gray_img > 100] = 255
gray_img[gray_img <= 100] = 0
black_padding = np.zeros((50, width))
gray_img = np.row_stack((black_padding, gray_img))

# kernel = np.ones((5, 5), np.uint8)
# # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 50))  # Adjust kernel size as needed
# closing = cv2.morphologyEx(gray_img, cv2.MORPH_CLOSE, kernel)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))  # Adjust kernel size as needed

# Perform opening operation (dilation followed by erosion) iteratively
num_iterations = 2  # Number of iterations
opening = cv2.morphologyEx(gray_img, cv2.MORPH_OPEN, kernel, iterations=num_iterations)

edges = cv2.Canny(cv2.convertScaleAbs(opening), 100, 200,apertureSize=7)

# cv2.imshow('edges', edges)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite(f"Testset/edges{name}.jpg",edges)

File name: Test7


True

In [60]:
import cv2
import numpy as np

# Load image, grayscale, Gaussian blur, threshold
image = cv2.imread("Testset/Test4.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.threshold(blur, 220, 255, cv2.THRESH_BINARY_INV)[1]

# Find contours
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
c = max(cnts, key=cv2.contourArea)

# Obtain outer coordinates
left = tuple(c[c[:, :, 0].argmin()][0])
right = tuple(c[c[:, :, 0].argmax()][0])
top = tuple(c[c[:, :, 1].argmin()][0])
bottom = tuple(c[c[:, :, 1].argmax()][0])

# Draw dots onto image
cv2.drawContours(image, [c], -1, (36, 255, 12), 2)
cv2.circle(image, left, 8, (0, 50, 255), -1)
cv2.circle(image, right, 8, (0, 255, 255), -1)
cv2.circle(image, top, 8, (255, 50, 0), -1)
cv2.circle(image, bottom, 8, (255, 255, 0), -1)

print('left: {}'.format(left))
print('right: {}'.format(right))
print('top: {}'.format(top))
print('bottom: {}'.format(bottom))
cv2.imshow('thresh', thresh)
cv2.imshow('image', image)
cv2.waitKey()

left: (0, 0)
right: (1279, 1282)
top: (0, 0)
bottom: (0, 1282)


-1

In [63]:
from PIL import Image, ImageFilter
 
img = Image.open(r"Testset/Test7.png")
 
# Converting the image to grayscale, as Sobel Operator requires
# input image to be of mode Grayscale (L)
img = img.convert("L")
 
# Calculating Edges using the passed laplacian Kernel
final = img.filter(ImageFilter.Kernel((3, 3), (-1, -1, -1, -1, 8,
                                          -1, -1, -1, -1), 1, 0))
 
final.save("Testset/EDGE_sample.png")